In [3]:
from sshtunnel import SSHTunnelForwarder 
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from dotenv import load_dotenv
import os
import pandas as pd

In [4]:
load_dotenv()

True

Need to install MariaDB Connector/C from CS Package Repository https://mariadb.com/docs/server/connect/programming-languages/c/install/

Need to install MariaDB community server on the machine

In [ ]:
with SSHTunnelForwarder(
    (os.getenv("BASTION_SERVER_IP"), 22), #Remote server IP and SSH port
    ssh_username = os.getenv("SSH_USER_NAME"),
    ssh_pkey=os.getenv("SSH_PRIVATE_KEY_PATH"),
    remote_bind_address=(os.getenv("RDS_ENDPOINT"), 3306)) as server: 
        
    server.start() #start ssh sever
    print('Server connected via SSH')
    
    #connect to PostgreSQL
    local_port = str(server.local_bind_port)
    database_name = 'financial'
    engine = create_engine(
        'mariadb+mariadbconnector://' 
        + os.getenv('DB_USERNAME') 
        + ':' 
        + os.getenv('DB_PASSWORD') 
        + '@127.0.0.1:'
        + local_port 
        + '/' 
        + database_name
    )

    Session = sessionmaker(bind=engine)
    session = Session()
    
    print('Database session created')
    
    #test data retrieval
    test = session.execute(text("SELECT * FROM trans LIMIT 10"))
    df = pd.DataFrame.from_records(test)
        
    session.close()

Server connected via SSH
Database session created


In [13]:
df.head()

,0,1,2,3,4,5,6,7,8,9
0,1,1,1995-03-24,PRIJEM,VKLAD,1000,1000,None,None,NaN
1,5,1,1995-04-13,PRIJEM,PREVOD Z UCTU,3679,4679,None,AB,41403269.0
2,6,1,1995-05-13,PRIJEM,PREVOD Z UCTU,3679,20977,None,AB,41403269.0
3,7,1,1995-06-13,PRIJEM,PREVOD Z UCTU,3679,26835,None,AB,41403269.0
4,8,1,1995-07-13,PRIJEM,PREVOD Z UCTU,3679,30415,None,AB,41403269.0
